# Set up:
Import libraries, config and get access token from refresh token

In [56]:
import yaml
import os
import requests
import numpy as np
import yaml
import datetime
import math

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) # prevents warnings after calls to Strava       

### Load Config
outside repo as conifential

In [88]:
with open('../auto_commute_config_secret.yaml', 'r') as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

### Set Up Commuication with Strava

In [61]:
auth_url = "https://www.strava.com/oauth/token"
get_activities_url = "https://www.strava.com/api/v3/athlete/activities"
activity_root_url = "https://www.strava.com/api/v3/activities/"
athlete_details_url = "https://www.strava.com/api/v3/athletes/" + str(config['athlete']['id']) + "/stats"

payload = {
    'client_id':'48638',
    'client_secret': config['api']['client_secret'],
    'refresh_token': config['api']['refresh_token'],
    'grant_type':"refresh_token",
    'f':'json'
}

print("Requesting Token...\n")

res = requests.post(auth_url, data = payload, verify = False)

access_token = res.json()['access_token']

# This is the header used in later api calls
header = {'Authorization': 'Bearer ' + access_token}

Requesting Token...



### Define helper functions 

In [49]:
def set_commute(activity_id, api_header):
    
    activity_update_url = activity_root_url + str(activity_id)
                                             
    results = requests.put(activity_update_url, headers = api_header, params = {'commute':'true'})    

    return results

In [31]:
def calculate_coord_distance(long_lat1, long_lat2): #inputs are numpy arrays
    
    R = 6371 # killo metres
    phi_1 = long_lat1[1] * math.pi/180 # φ, λ in radians
    phi_2 = long_lat1[1] * math.pi/180
    del_phi = (long_lat1[1]-long_lat2[1]) * math.pi/180
    del_lambda = (long_lat1[0]- long_lat2[0]) * math.pi/180

    a = math.sin(del_phi/2) * math.sin(del_phi/2) + math.cos(phi_1) * math.cos(phi_2) * math.sin(del_lambda/2) * math.sin(del_lambda/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    return R * c #// in kilometres

calculate_coord_distance(np.array([0,0]),np.array([0,1]))

111.19492664455873

In [29]:
def get_pcode_centre(postcode):
       
    get_pcode_centre_url = 'http://api.postcodes.io/postcodes/' + postcode
    
    pcode_req = requests.get(get_pcode_centre_url).json()
    
    long = float(pcode_req['result']['longitude'])
    lat = float(pcode_req['result']['latitude'])
    
    return np.array([long,lat])

get_pcode_centre('SW1A1AA')

array([-0.141588, 51.501009])

In [34]:
def check_commute_match(start_longlat, end_longlat, check_longlat1, check_longlat2, sensitivity):
    # check start is close enough to either of points
    #   if the start matches point 1, check end matches point 2
    #   if the start matches point 2, check end matches point 1
    #   cannot simply check if start and end match either as eitherwise loops at any of the poitns would match
    if calculate_coord_distance(start_longlat, check_longlat1) < sensitivity:
        if calculate_coord_distance(end_longlat, check_longlat2) < sensitivity:
            return True
    
    if calculate_coord_distance(start_longlat, check_longlat2) < sensitivity :
        if calculate_coord_distance(end_longlat, check_longlat1) < sensitivity:
            return True

    return False

check_commute_match(np.array([-0.416652, 51.332115]),np.array([-0.175761, 51.456331]),np.array([-0.175761, 51.456331]),np.array([-0.416652, 51.332112]),2)

True

In [79]:
def check_activity_for_commute(activity, commute_centroids, verbose = True):
    
    try: 
        start_longlat = np.array([activity[ 'start_longitude'] , activity['start_latitude'] ])
        end_longlat = np.array([ activity['end_latlng'][1] , activity['end_latlng'][0] ])
    except: 
        print(str(activity['name']) + ' --- could not retreive GPS details')
        return ''
    
    for commute in commute_centroids:
        if check_commute_match(start_longlat, end_longlat, commute_centroids[commute][0], commute_centroids[commute][1] , 1 ):
            return activity['name'] + ' --- ' + str(activity['start_date_local']) + ' --- Positive Match to: ' + str(commute)

    return ''

# Get Data Required for Process

### Retreive Strava Data
Future improvement - return all activities rather than just x most recent

In [65]:
athlete_info = requests.get(athlete_details_url, headers = header).json()
print(yaml.dump(athlete_info))
athelete_total_activities = athlete_info['all_ride_totals']['count'] + athlete_info['all_run_totals']['count'] + athlete_info['all_swim_totals']['count']

def retreive_activities(per_page=200, page=1):

    param = {'per_page': per_page, 'page': page} #some large number hoping to return all

    return requests.get(get_activities_url, headers = header, params = param).json()

activities_page_one = retreive_activities()

print("Retrieved " + str(len(activities_page_one)) + " activities")

all_ride_totals:
  count: 1271
  distance: 35310291
  elapsed_time: 7728573
  elevation_gain: 227995
  moving_time: 5099658
all_run_totals:
  count: 30
  distance: 160101
  elapsed_time: 61069
  elevation_gain: 652
  moving_time: 52779
all_swim_totals:
  count: 0
  distance: 0
  elapsed_time: 0
  elevation_gain: 0
  moving_time: 0
biggest_climb_elevation_gain: 491.7
biggest_ride_distance: 333422.0
recent_ride_totals:
  achievement_count: 484
  count: 23
  distance: 496485.5087890625
  elapsed_time: 454055
  elevation_gain: 3428.9885473251343
  moving_time: 73284
recent_run_totals:
  achievement_count: 0
  count: 0
  distance: 0.0
  elapsed_time: 0
  elevation_gain: 0.0
  moving_time: 0
recent_swim_totals:
  achievement_count: 0
  count: 0
  distance: 0.0
  elapsed_time: 0
  elevation_gain: 0.0
  moving_time: 0
ytd_ride_totals:
  count: 125
  distance: 3478703
  elapsed_time: 1029316
  elevation_gain: 22034
  moving_time: 509866
ytd_run_totals:
  count: 5
  distance: 22212
  elapsed_tim

Print out information on longest ago activity

In [40]:
first_activity = activities_all[len(activities_all)-1]

print(first_activity['name'] + ':\n---\n')
print(yaml.dump(first_activity))

Morning Ride:
---

achievement_count: 1
athlete:
  id: 10596968
  resource_state: 1
athlete_count: 2
average_cadence: 88.2
average_heartrate: 133.6
average_speed: 7.134
average_temp: 11
average_watts: 133.3
comment_count: 0
commute: false
device_watts: true
display_hide_heartrate_option: true
distance: 30054.8
elapsed_time: 4397
elev_high: 53.8
elev_low: 25.2
end_latlng:
- 51.354113
- -0.463409
external_id: garmin_push_4098747837
flagged: false
from_accepted_tag: false
gear_id: null
has_heartrate: true
has_kudoed: false
heartrate_opt_out: false
id: 2741867714
kilojoules: 561.6
kudos_count: 0
location_city: null
location_country: United Kingdom
location_state: null
manual: false
map:
  id: a2741867714
  resource_state: 2
  summary_polyline: iiayH|bc@p@fIzApMd@~BZpEDfCAtGYnDo@pDCzBUdCIfBJxBp@nJXxFLzHRnE@z@GdBWvAS`EWrB]bAcCpEc@`AuBnGeC~J}@dF?rAMf@KbAYnAk@fES|Bs@vDs@~EKzABXFHMt@c@nAaDxQeC`I[pASvA[vEWpJa@fHmB`UaAfKs@nEe@pE}@zLShGWhCS`DOrDCdDYlIAlGBjCTlGFfFBPDNXPbEpBtElCdHrD|NpI|OjI|DdCbB~@z

### Convert Address into GPS Coordinates

In [89]:
# Create a copy of the commutes specificaiton, replacing postcodes with gps centroids
def convert_commutes_specs(commutes_dict):

    commute_points = commutes_dict

    for commute in config['commutes']:
        #print(commute)
        #print(config['commutes'][commute])
        #print(type(config['commutes'][commute]))
        #print(len(config['commutes'][commute]))
        commute_points[commute] = list(map(get_pcode_centre,config['commutes'][commute]))
        
    return commute_points
           
commutes_pcodes_centroids = convert_commutes_specs(config['commutes'])

# Run routine

In [90]:
def update_activities(activities):

    run_api_calls = 0
    run_matches = 0

    for activity_i in activities:
        result = check_activity_for_commute(activity = activity_i, commute_centroids = commutes_pcodes_centroids)
        
        if len(result) > 1:
            
            run_matches += 1
            
            if not activity_i['commute']:     
                print(result)
                run_api_calls += 1
                set_commute(activity_i['id'], api_header = header )
                print('--- Activity id: ' + str(activity_i['id']) + ' set to commute')

        #else:
        #    print('No match for ' + activity_i['name'])

        #if api_call_count > 100:
        #    break

    print('\n\n Finished loop:' + str(run_api_calls) + ' out of ' + str(run_matches) + ' matches modified (' + str(len(activities)) + ' were checked)')
    
    return(run_api_calls)

In [92]:
changes = update_activities(activities_page_one)

Missing milles --- could not retreive GPS details
Big half, easier by bike --- 2020-03-01T09:27:39Z --- Positive Match to: house_two_to_work_location_three
--- Activity id: 3145532193 set to commute
Lunch Ride --- could not retreive GPS details
Afternoon Ride --- 2020-01-09T17:28:19Z --- Positive Match to: house_two_to_work_location_three
--- Activity id: 3002085684 set to commute
Morning Ride --- 2020-01-09T08:10:05Z --- Positive Match to: house_two_to_work_location_three
--- Activity id: 3002084978 set to commute
Evening Ride --- could not retreive GPS details
Morning Ride --- 2020-01-06T08:17:10Z --- Positive Match to: house_two_to_work_location_three
--- Activity id: 2990483034 set to commute
Evening Ride --- could not retreive GPS details


 Finished loop:4 out of 92 matches modified (200 were checked)


In [98]:
total_pages = math.ceil(athelete_total_activities / 200)
print(total_pages)

if total_pages > 1:
    for i in range(2, total_pages + 1):
        print(i)
        activities_page_i = retreive_activities(page=i)
        
        changes = update_activities(activities_page_i)
        print(changes)
        
        if changes == 0:
            break

7
2


TypeError: string indices must be integers